# Neural Networks

To make a neural network using Keras first you create an instance of the Sequential class, then you call add on the Sequential object to add layers. The layers are instances of the Dense class which represents a fully connected layer with a specified number of neurons.

In [7]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

model = Sequential()
model.add(Dense(3, activation='relu',input_dim=2))
model.add(Dense(1))

When all the layers are added the next step is to compile and specify attributes such as which optimizer and loss function to use during training.

In [10]:
model.compile(optimizer='adam', loss='mae', metrics=['mae'])

loss = 'mae' means that keras will use mean absolute error to measure loss

metrics = ['mae'] means that keras will capture MAE values and use it after training is complete to judge the efficacy of the training

In [12]:
#hist = model.fit(x,y, epochs=100, batch_size=100, validation_split=0.2)

The fit method returns a history object containing the training and validation metrics specified in the metrics parameter passed to the compile method, for example metrics=['mae'] captures MAE at the end of each epoch. The greater the gap between the training and validation accuracy, the greater the likelihood that the network is overfitting.

In [13]:
#prediction = mode.predict(np.array([[2,2]])

## Sizing a Neural Network

A neural network is defined by the number of layers (the depth), the number of neurons in each layer (the width of the layeres), the types of layers, and the activation functions used. When designing a network it is important that you pick the minimum width and depth which achives the accuracy required. There are a few guidelines when making this choice:
1. Greater widths and depths give the network more capacity to "learn" by fitting more tightly to the training data but also increase the risk of overfitting. The simplest way to loosen the fit is to reduce the number of neurons.
2. Generarly it is better to have greater width than greater depth to avoid the vanishing gradient problem.
3. Fewer neurons means less training time